In [ ]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Establish client connection
client = MongoClient('mongodb://localhost:27017')
db = client['university_db']
courses_collection = db['courses']

# Bulk insert of courses with student enrollments
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print('Courses inserted successfully.')


Courses inserted successfully.


In [3]:
# Query for courses with enrollments greater than 20
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

{'_id': ObjectId('675c937b667ecb43fb3b15ff'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('675c937b667ecb43fb3b1600'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [4]:
# Query courses in Computer Science or Mathematics departments
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

{'_id': ObjectId('675c937b667ecb43fb3b15ff'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('675c937b667ecb43fb3b1600'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [5]:
# Average enrollment per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'Computer Science', 'average_enrollment': 25.0}
{'_id': 'History', 'average_enrollment': 20.0}
{'_id': 'Physics', 'average_enrollment': 15.0}
{'_id': 'Mathematics', 'average_enrollment': 30.0}


In [6]:
# Maximum enrollment per department
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Mathematics', 'max_enrollment': 30}
{'_id': 'History', 'max_enrollment': 20}
{'_id': 'Physics', 'max_enrollment': 15}
{'_id': 'Computer Science', 'max_enrollment': 25}


In [7]:
# Projection to rename fields
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': ObjectId('675c937b667ecb43fb3b15ff'), 'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('675c937b667ecb43fb3b1600'), 'enrollments': 25, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('675c937b667ecb43fb3b1601'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'_id': ObjectId('675c937b667ecb43fb3b1602'), 'enrollments': 15, 'course_name': 'Physics 202', 'department_name': 'Physics'}


In [8]:
# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': ObjectId('675c937b667ecb43fb3b15ff'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('675c937b667ecb43fb3b1600'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('675c937b667ecb43fb3b1601'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'_id': ObjectId('675c937b667ecb43fb3b1602'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}


# Homework

In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017')
db = client['university_db']
coursesCollection = db['courses']
studentsCollection = db['students']

# Tugas 1
pipelineHw1 = [
    {'$group': {'_id': '$department', 'course_count': {'$sum': 1}}}
]
print("\nTugas 1")
for result in coursesCollection.aggregate(pipelineHw1):
    print(result)

# Tugas 2
pipelineHw2 = [
    {'$match': {'enrollments': {'$gt': 25}, 'department': 'Computer Science'}},
    {'$group': {'_id': '$department', 'courses_over_25': {'$push': '$course'}}}
]
print("\nTugas 2")
for result in coursesCollection.aggregate(pipelineHw2):
    print(result)

# Tugas 3
pipelineHw3 = [
    {'$lookup': {
        'from': 'students',
        'localField': 'course',
        'foreignField': 'course_name',
        'as': 'student_details'
    }}
]
print("\nTugas 3")
for result in coursesCollection.aggregate(pipelineHw3):
    print(result)



Tugas 1
{'_id': 'Mathematics', 'course_count': 1}
{'_id': 'History', 'course_count': 1}
{'_id': 'Physics', 'course_count': 1}
{'_id': 'Computer Science', 'course_count': 1}

Tugas 2

Tugas 3
{'_id': ObjectId('675c937b667ecb43fb3b15ff'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_details': []}
{'_id': ObjectId('675c937b667ecb43fb3b1600'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'student_details': []}
{'_id': ObjectId('675c937b667ecb43fb3b1601'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'student_details': []}
{'_id': ObjectId('675c937b667ecb43fb3b1602'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'student_details': []}
